In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
# Libraries
%load_ext autoreload
%autoreload 2
import index_toolkit as tk
import pandas as pd
import yfinance as yf

In [3]:
# Inputs
date_input = '13/09/2024'
df_market_input = '^FCHI'
riskfree_input = '^TNX'
df_allocation = pd.read_csv('index_allocation.csv', index_col = 'Ticker')

In [4]:
# Fetching df_market data and risk-free rate
df_market = yf.Ticker(df_market_input).history('max')['Close']
df_market.name = df_market_input
df_market.index = df_market.index.tz_localize(None)
riskfree = yf.Ticker(riskfree_input).history('1y')['Close'].iloc[-1] / 100

# Fetching and reating a DataFrame to store historical prices for each asset
df_history = pd.DataFrame()
for ticker in df_allocation.index:
    history = yf.Ticker(ticker).history('max')['Close']
    history.name = ticker
    df_history = pd.concat([df_history, history], axis=1)
df_history.index = pd.to_datetime(df_history.index)
df_history = df_history.resample('B').last().ffill()
df_history.index = df_history.index.tz_localize(None)

# Calculating asset valuations and allocation
class index_table(BaseClass):
    def __init__(self, history, allocation, date, by=None):
        super().__init__(date)
        # Assigning input parameters to instance variables
        self.history = history
        self.date = self.get_date()
        self.allocation = allocation
        self.dict_by = by

    def compute(self):
        index = pd.DataFrame()
        index.index = self.allocation.index
        index['Price'] = self.history.loc[self.date]
        index['Share'] = self.allocation['Share']
        index['Valorisation'] = self.allocation['Share'] * index['Price']
        index['Allocation'] = index['Valorisation'] / index['Valorisation'].sum()
        index['Date'] = self.date
        
        if self.dict_by:
            for by in self.dict_by:
                index[by] = self.allocation[by]
            for by in self.dict_by:
                by_index = index.groupby(by)['Allocation'].transform('sum')
                index[f'Alloc. by {by}'] = index['Allocation'] / by_index
                
        return index

def group_history(self, by):
    if by == 'All':
        return self.history
    elif by == 'Overall':
        history = (self.index['Allocation'] * self.history.pct_change()).T.sum()
        history = 100 * (history + 1).cumprod()
        history.name = 'Overall'
        history = pd.DataFrame(history)
        return history
    else:
        #history = (self.index[f'Alloc. by {by}'] *self.history.pct_change()).T
        return self.history

def show(self):
    display = self.compute()
    display = display[['Price', 'Share', 'Valorisation', 'Allocation']]
    display['Allocation'] = display['Allocation'].apply(lambda x: f"{x * 100:.2f}%")

    with pd.option_context(
        'display.max_rows', None,
        'display.max_columns', None,
        'display.float_format', '{:,.1f}'.format,
        'display.max_colwidth', None,
        'display.colheader_justify', 'center'
    ):
        print(display)

In [10]:
# Statistiques du portefeuille par positions
tk.stats_table(
    history=df_history, 
    market=df_market, 
    riskfree=riskfree, 
    index=df_allocation, 
    date=date_input
).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…

In [6]:
tk.chart_comparison(
    history=df_history, 
    date=date_input
).show()

AttributeError: 'chart_comparison' object has no attribute 'get_range_keys'

In [13]:
tk.top_n_flop(
    history=df_history, 
    date=date_input
).show()

interactive(children=(Select(description='range', options=('1W', '1M', 'YTD', '1Y', '5Y', '10Y'), value='1W'),…